Lê os arquivos dos anos de 2021, 2022, 2023 e 2024 a respeito do comportamento de cada contentor em cada dia entre janeiro e abril

In [1]:
import pandas as pd

# Caminhos dos arquivos enviados
file_2021 = "resultado_contentores_2021.xlsx"
file_2022 = "resultado_contentores_2022.xlsx"
file_2023 = "resultado_contentores_2023.xlsx"
file_2024 = "resultado_contentores_2024.xlsx"

# Carregar os dados dos arquivos Excel
df_2021 = pd.read_excel(file_2021)
df_2022 = pd.read_excel(file_2022)
df_2023 = pd.read_excel(file_2023)
df_2024 = pd.read_excel(file_2024)


Calcula as medidas como média de enchimento e desvio padrão dos anos 2021, 2022 e 2023

In [2]:
import numpy as np

# Unificar os dados de 2021 a 2023 para estatísticas
df_hist = pd.concat([df_2021, df_2022, df_2023])

# Converter para formato longo para facilitar o cálculo de estatísticas
df_hist_long = df_hist.melt(id_vars='idcontentor', var_name='data', value_name='enchimento')

# Calcular média e desvio padrão por contentor
stats = df_hist_long.groupby('idcontentor')['enchimento'].agg(['mean', 'std']).reset_index()
stats.columns = ['idcontentor', 'media_enchimento', 'desvio_padrao']
stat_path = "estat_id.xlsx"
stats.to_excel(stat_path, index=False)
# Mostrar os primeiros resultados
# stats.head(400)


Usar a próxima celula para criar um ambiente no gurobi para acelerar o processo

In [3]:
from gurobipy import Env

# Cria um ambiente do Gurobi (com log desativado se quiser)
env = Env(empty=True)
env.setParam("LogFile", "gurobi_vrpp.log")
env.start()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2629153
Set parameter LogFile to value "gurobi_vrpp.log"
Academic license 2629153 - for non-commercial use only - registered to yg___@tecnico.ulisboa.pt


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=2629153, LogFile=gurobi_vrpp.log>

Constroi a lógica da heuristica do Look-ahead para o ano de 2024 usando os dados de 2021 2022 e 2023.

In [4]:
import numpy as np
import os
import pickle
import pandas as pd
# from VRPP import VRPP
# from VRPP_joana import VRPP

import signal

parar_otimizacao = False

def signal_handler(sig, frame):
    global parar_otimizacao
    parar_otimizacao = True
    print("\n Interrupção manual detectada. Parando o solver...")

signal.signal(signal.SIGINT, signal_handler)

def meu_callback(model, where):
    global parar_otimizacao
    if where == GRB.Callback.MIP:
        if parar_otimizacao:
            print("\nParando o solver e pegando a melhor solução encontrada...")
            model.terminate()

import os
import folium
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB, quicksum
import time


def extrair_lat_lon_dict(arquivo_latlong):
    df = pd.read_excel(arquivo_latlong)
    return {
        f"{float(row['idcontentor']):.1f}": (row['Latitude'], row['Longitude'])
        for _, row in df.iterrows()
    }

def plotar_mapa_rota(rota, id_map, idx_rota, caminho_saida, latlon_dict, MG):
    pontos_rota = []
    for i, j in rota:
        id_j = f"{float(id_map[i]):.1f}"
        if id_j in latlon_dict:
            pontos_rota.append(latlon_dict[id_j])

    if pontos_rota:
        mapa = folium.Map(location=pontos_rota[0], zoom_start=12)

        for idx, (lat, lon) in enumerate(pontos_rota):
            if idx == 0:
                cor = "black"  # depósito
            else:
                id_original = id_map[rota[idx - 1][1]]  # extrai ID real do contentor
                cor = "purple" if id_original in MG else "blue"

            folium.Marker(
                [lat, lon],
                popup=f"Ponto {idx + 1}",
                tooltip=f"Ponto {idx + 1}",
                icon=folium.Icon(color=cor)
            ).add_to(mapa)

        pontos_rota.append(pontos_rota[0])  # volta ao depósito
        folium.PolyLine(pontos_rota, color="blue", weight=2.5, opacity=1).add_to(mapa)

        nome_arquivo = os.path.join(caminho_saida, f"mapa_rota_{idx_rota}.html")
        mapa.save(nome_arquivo)

def ler_matriz_de_excel():
    caminho_arquivo_matriz = 'C:/Users/ygora/Documents/IC-Tecnico/cenário3/Embalagens de papel e cartão/matriz_distancia_papel_com_base.xlsx'
    df = pd.read_excel(caminho_arquivo_matriz, index_col=0)
    ids_principais = list(df.index)
    matriz = df.to_numpy()
    return matriz, ids_principais

def extrair_submatriz(matriz_principal, ids_principais, ids_rota):
    indices = [ids_principais.index(id_) for id_ in ids_rota]
    submatriz = matriz_principal[np.ix_(indices, indices)]
    return submatriz

def VRPP(contentores_df, dia_coleta, env=None, MG=[]):

    latlon_dict = extrair_lat_lon_dict('Lat_Long.xlsx')

    ids_rota = contentores_df['idcontentor'].tolist()
    enchimentos = contentores_df['Enchimento'].tolist()
    criticos = [bool(val) for val in contentores_df['Status'].tolist()]
    print("A quantidade de contentores MG",sum(criticos) - 1)

    matriz_completa, ids_principais = ler_matriz_de_excel()
    if not all(id_ in ids_principais for id_ in ids_rota):
        raise ValueError("Alguns IDs do Excel não estão na matriz de distância.")
    submatriz = extrair_submatriz(matriz_completa, ids_principais, ids_rota)

    B, V, Q = 30, 2.5, 2200 #densidade, volume, capacidade
    R, C, Omega = 0.121, 1, 0.1 #receita, custo, omega
    delta, psi = 0, 1 #delta, psi

    pesos_reais = [(e / 100) * B * V for e in enchimentos]
    nodes = list(range(len(ids_rota)))
    idx_deposito = 0
    nodes_real = [i for i in nodes if i != idx_deposito]
    S_dict = {i: pesos_reais[i] for i in nodes}
    criticos_dict = {i: criticos[i] for i in nodes}

    max_dist = 6000
    pares_viaveis = [(i, j) for i in nodes for j in nodes if i != j and submatriz[i][j] <= max_dist]

    mdl = gp.Model("VRPP", env=env) if env else gp.Model("VRPP")

    x = mdl.addVars(pares_viaveis, vtype=GRB.BINARY, name="x") #diz se a gente usa ou não a estrada que vai do ponto i até o ponto j
    y = mdl.addVars(pares_viaveis, vtype=GRB.CONTINUOUS, lb=0, name="y") # quanto de resíduo (kg) a gente está carregando nesse trecho entre i e j
    f = mdl.addVars(pares_viaveis, vtype=GRB.CONTINUOUS, lb=0, name="f") # pra evitar que o modelo crie "ciclos pequenos" fora do caminho principal (subtours).
    g = mdl.addVars(nodes, vtype=GRB.BINARY, name="g")
    k_var = mdl.addVar(lb=0, vtype=GRB.INTEGER, name="k_var")


    for i, j in pares_viaveis:
        mdl.addConstr(y[i, j] <= Q * x[i, j]) # limita que o trecho não tenha a capaciade maxima do caminhão
        mdl.addConstr(f[i, j] <= len(nodes) * x[i, j]) #evita subtours

    # Garante que o fluxo líquido em cada nó é igual ao resíduo gerado somente se o contentor for coletado
    for i in nodes_real:
        mdl.addConstr(quicksum(y[i, j] - y[j, i] for j in nodes if (i, j) in y or (j, i) in y) == S_dict[i] * g[i])

    # Teste de fixar o valor da quantidade de caminhões
    MAX_TRUCKS = 4
    mdl.addConstr(k_var <= MAX_TRUCKS)


    # Relaciona k_var com o número de rotas que partem e voltam do depósito.
    mdl.addConstr(k_var == quicksum(x[idx_deposito, j] for j in nodes_real if (idx_deposito, j) in x))
    mdl.addConstr(quicksum(x[idx_deposito, j] for j in nodes_real if (idx_deposito, j) in x) == k_var)
    mdl.addConstr(quicksum(x[j, idx_deposito] for j in nodes_real if (j, idx_deposito) in x) == k_var)


    # Se um contentor não for coletado (g[j]==0), não pode haver rota conectando-o ao depósito.
    for j in nodes_real:
        if (idx_deposito, j) in x: mdl.addConstr(x[idx_deposito, j] <= g[j])
        if (j, idx_deposito) in x: mdl.addConstr(x[j, idx_deposito] <= g[j])


    #Garante que pelo menos um número mínimo de contentores críticos seja visitado (ajustável por delta)
    mdl.addConstr(quicksum(g[i] for i in nodes_real if criticos_dict[i]) >= len([i for i in nodes_real if criticos_dict[i]]) - len(nodes_real) * delta)

    for i in nodes_real:
        # Esses contentores devem ser coletados.
        if criticos_dict[i] or enchimentos[i] >= psi * 100:
            mdl.addConstr(g[i] == 1)
        # g[i] é forçado a ser 0 (não coleta).
        if enchimentos[i] < 10 and not criticos[i]:
            g[i].ub = 0

    # Se um contentor for visitado (g[j]==1), deve ter exatamente uma entrada e uma saída.
    for j in nodes_real:
        mdl.addConstr(quicksum(x[i, j] for i in nodes if (i, j) in x) == g[j])
        mdl.addConstr(quicksum(x[j, k] for k in nodes if (j, k) in x) == g[j])


    # Assegura conectividade entre os pontos e impede a criação de ciclos menores isolados (subtours).
    mdl.addConstr(quicksum(f[0, j] for j in nodes_real if (0, j) in f) == quicksum(g[j] for j in nodes_real))
    for j in nodes_real:
        mdl.addConstr(quicksum(f[i, j] for i in nodes if (i, j) in f) - quicksum(f[j, k] for k in nodes if (j, k) in f) == g[j])

    mdl.setObjective(
        R * quicksum(S_dict[i] * g[i] for i in nodes_real)
        - 0.5 * C * quicksum(x[i, j] * submatriz[i][j] for i, j in pares_viaveis)
        - Omega * k_var,
        GRB.MAXIMIZE
    )


    mdl.Params.MIPFocus = 1
    mdl.Params.Heuristics = 0.5
    mdl.Params.Threads = 0
    mdl.Params.Cuts = 3
    mdl.Params.CliqueCuts = 2    # força clique cuts
    mdl.Params.CoverCuts = 2     # força cuts de conjuntos
    mdl.Params.FlowCoverCuts = 2 # força cortes para fluxos
    mdl.Params.GUBCoverCuts = 2
    mdl.Params.Presolve = 1
    mdl.Params.NodefileStart = 0.5
    mdl.setParam("MIPGap", 0.05)
    mdl.Params.TimeLimit = 14400

    global parar_otimizacao
    parar_otimizacao = False

    start = time.time()
    mdl.optimize(meu_callback)
    end = time.time()
    tempo = end - start

    contentores_coletados = []

    if mdl.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.INTERRUPTED]:
        resultados_y = []
        resultados_g = []
        id_map = {i: ids_rota[i] for i in nodes}
        arcos_ativos = [(i, j) for i in nodes for j in nodes if i != j and x[i, j].X > 0.5]
        final_gap = mdl.MIPGap

        rotas = []
        visitados = set()
        while True:
            rota = []
            atual = 0
            while True:
                prox = [j for (i, j) in arcos_ativos if i == atual and (i, j) not in visitados]
                if not prox:
                    break
                j = prox[0]
                visitados.add((atual, j))
                rota.append((atual, j))
                atual = j
                if j == 0:
                    break
            if rota:
                rotas.append(rota)
            else:
                break

        for i in nodes:
            for j in nodes:
                if i != j and y[i, j].X > 0:
                    resultados_y.append((id_map[i], id_map[j], y[i, j].X))

        # Variáveis g[i]
        for i in nodes:
            if g[i].X > 0.5:
                resultados_g.append((id_map[i], g[i].X))

        k_value = int(k_var.X)
        total_residuos = sum(S_dict[i] * g[i].X for i in nodes_real if g[i].X > 0.5)
        distancia_total = sum(submatriz[i][j] * x[i, j].X * 0.5 for i in nodes for j in nodes if i != j and x[i, j].X > 0.5)
        ratio = distancia_total / total_residuos if total_residuos > 0 else 0
        taxa_uso_veiculo = (total_residuos / (k_value * Q)) * 100 if k_value > 0 else 0
        bins_visitados = len([i for i in nodes_real if g[i].X > 0.5])
        criticos_atendidos = len([i for i in nodes_real if g[i].X > 0.5 and criticos_dict[i]])

        caminho_saida = os.path.join(f"Resultados/{dia_coleta.strftime('%Y-%m-%d')}")
        os.makedirs(caminho_saida, exist_ok=True)

        with pd.ExcelWriter(os.path.join(caminho_saida, 'resultado.xlsx')) as writer:
            # Cada rota vira uma planilha separada
            for idx, rota in enumerate(rotas, start=1):
                df_rota = pd.DataFrame(
                    [(id_map[i], id_map[j], x[i, j].X) for (i, j) in rota],
                    columns=['i', 'j', 'x_ij']
                )
                df_rota.to_excel(writer, sheet_name=f'Rota_{idx}', index=False)
                plotar_mapa_rota(rota, id_map, idx, caminho_saida, latlon_dict, MG)
                contentores_coletados.extend([id_map[j] for (i, j) in rota if j != 0])

            # Salvar outras variáveis
            df_y = pd.DataFrame(resultados_y, columns=['i', 'j', 'y_ij'])
            df_g = pd.DataFrame(resultados_g, columns=['idContentor', 'g_i'])
            df_k = pd.DataFrame([k_value], columns=['k_var'])

            df_y.to_excel(writer, sheet_name='Y_Variables', index=False)
            df_g.to_excel(writer, sheet_name='G_Variables', index=False)
            df_k.to_excel(writer, sheet_name='K_Variable', index=False)

            kpi_rotas = []
            for idx, rota in enumerate(rotas, start=1):
                residuos = sum(S_dict[j] for (i, j) in rota if j != 0 and g[j].X > 0.5)
                distancia = sum(submatriz[i][j] * 0.5 for (i, j) in rota)  # ida e volta contada como .5
                ratio_waste = residuos / distancia if distancia > 0 else 0
                ratio_distance =  distancia / residuos if residuos > 0 else 0
                uso_veic = (residuos / Q) * 100 if Q > 0 else 0
                kpi_rotas.append({
                    'Caminhão': f'Rota_{idx}',
                    'Resíduos (kg)': residuos,
                    'Distância (km)': distancia,
                    'Ratio (kg/km)': ratio_waste,
                    'Ratio (km/Kg)': ratio_distance,
                    'Uso do Veículo (%)': uso_veic
                })

            df_kpi = pd.DataFrame({
                'KPI': [
                    'Objective Value', 'Solution Time', 'Gap final', 'R', 'Q', 'Omega',
                    'Total Waste Collected', 'Total Distance',
                    'Bins Visited', 'Critical Bins Visited',
                    'Ratio (km/Kg)', 'Vehicles Used', 'Vehicle Usage Rate_(%)'
                ],
                'Value': [
                    mdl.objVal, tempo, final_gap, R, Q, Omega,
                    total_residuos, distancia_total,
                    bins_visitados, criticos_atendidos,
                    ratio, k_value, taxa_uso_veiculo
                ]
            })

            df_kpi_individual = pd.DataFrame(kpi_rotas)

            # Escreve ambas no Excel
            df_kpi.to_excel(writer, sheet_name='KPI_Geral', index=False)
            df_kpi_individual.to_excel(writer, sheet_name='KPI_Rotas', index=False)
            
            if mdl.status == GRB.TIME_LIMIT:
                mdl.write(os.path.join(caminho_saida, f"modelo_parcial_{dia_coleta.strftime('%Y-%m-%d')}.lp"))
                mdl.write(os.path.join(caminho_saida, f"solucao_parcial_{dia_coleta.strftime('%Y-%m-%d')}.mst"))

    return contentores_coletados











# Parâmetros
Z = 0
dias = df_2024.columns[1:]  # datas de 2024
ids = df_2024['idcontentor'].values
stats_dict = stats.set_index('idcontentor').to_dict()

if os.path.exists('checkpoint.pkl'):
    with open('checkpoint.pkl', 'rb') as f:
        data = pickle.load(f)
    t = data['t']
    nivel_atual = data['nivel_atual']
    dias_coleta = data['dias_coleta']
    must_go_por_dia = data['must_go_por_dia']
    historico_niveis = data['historico_niveis']
    print(f"Retomando a partir do dia {t + 1} ({dias[t]})")
else:
    t = 0
    nivel_atual = {cid: 0 for cid in ids}
    dias_coleta = []
    must_go_por_dia = []
    historico_niveis = []

while t < len(dias):    
    dia = dias[t]

    # Salvar checkpoint
    with open('checkpoint.pkl', 'wb') as f:
        pickle.dump({
            't': t,  # salva exatamente onde parou
            'nivel_atual': nivel_atual,
            'dias_coleta': dias_coleta,
            'must_go_por_dia': must_go_por_dia,
            'historico_niveis': historico_niveis
        }, f)

    print(f"A data em analise é {dia}")
    
    # Atualiza o nível atual com o enchimento do dia
    niveis_dia = {'data': dia}
    for i, cid in enumerate(ids):
        nivel_atual[cid] += df_2024.iloc[i, t + 1]
        niveis_dia[cid] = nivel_atual[cid]
    historico_niveis.append(niveis_dia)

    # Verifica se algum contentor vai estourar amanhã (t+1)
    MG = []
    for cid in ids:
        media = stats_dict['media_enchimento'].get(cid, 0)
        desvio = stats_dict['desvio_padrao'].get(cid, 0)

        if t + 1 >= len(dias):
            continue

        estimativa = nivel_atual[cid] + media * 1 + Z * desvio * np.sqrt(1)
        if estimativa >= 100:
            MG.append(cid)

    if not MG:
        t += 1
        continue

    # Descobrir o próximo nc (próxima coleta obrigatória)
    t_prime = t + 1
    while t_prime < len(dias):
        algum_estoura = False
        for cid in MG:
            media = stats_dict['media_enchimento'].get(cid, 0)
            desvio = stats_dict['desvio_padrao'].get(cid, 0)

            estimativa = 0 + media * (t_prime - t) + Z * desvio * np.sqrt(t_prime - t)
            if estimativa >= 100:
                algum_estoura = True
                break
        if algum_estoura:
            break
        t_prime += 1

    nc = t_prime  # Próximo dia crítico encontrado

    # Verificar se outros contentores não-MG vão estourar antes de nc
    for cid in ids:
        if cid in MG:
            continue
        media = stats_dict['media_enchimento'].get(cid, 0)
        desvio = stats_dict['desvio_padrao'].get(cid, 0)
        for k in range(1, nc - t):
            estimativa = nivel_atual[cid] + media * k + Z * desvio * np.sqrt(k)
            if estimativa >= 100:
                MG.append(cid)
                break

    # Criar DataFrame com os contentores
    contentores_df = pd.DataFrame({
        'idcontentor': ids,
        'Enchimento': [nivel_atual[cid] for cid in ids],
        'Status': [cid in MG for cid in ids]
    })

    # Adicionar o depósito no início do DataFrame
    deposito_df = pd.DataFrame([{
        'idcontentor': 0,
        'Enchimento': 0,
        'Status': True
    }])

    # Concatenar o depósito com os demais contentores
    contentores_df = pd.concat([deposito_df, contentores_df], ignore_index=True)
    
    dias_coleta.append(dias[t])
    must_go_por_dia.append(MG.copy())

    # VRPP do Hector
    coletados = VRPP(contentores_df, dias[t], env=env, MG=MG)
    
    # VRPP da Joana
    # coletados = VRPP(contentores_df, dias[t], number_vehicles=4, env=env, MG=MG)

    for cid in coletados:
        nivel_atual[cid] = 0

    t += 1  # só avança se tudo acima deu certo

# Criar DataFrames para exportação
df_dias = pd.DataFrame({'dia_coleta': dias_coleta})
df_must_go = pd.DataFrame({'dia_coleta': dias_coleta, 'contentores_must_go': must_go_por_dia})
df_niveis = pd.DataFrame(historico_niveis).set_index('data').T
df_niveis.insert(0, 'idcontentor', df_niveis.index)


Retomando a partir do dia 116 (2024-04-25 00:00:00)
A data em analise é 2024-04-25 00:00:00
A quantidade de contentores MG 62
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Threads to value 0
Set parameter Cuts to value 3
Set parameter CliqueCuts to value 2
Set parameter CoverCuts to value 2
Set parameter FlowCoverCuts to value 2
Set parameter GUBCoverCuts to value 2
Set parameter Presolve to value 1
Set parameter NodefileStart to value 0.5
Set parameter MIPGap to value 0.05
Set parameter TimeLimit to value 14400
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  14400
MIPGap  0.05
Heuristics  0.5
MIPFocus  1
NodefileStart  0.5
Cuts  3
CliqueCuts  2
CoverCuts  2
FlowCoverCuts  2
GUBCoverCuts  2
Presolve  1

Ac

Salva os dados do Look-ahead em um excel

In [5]:
must_go_dict = {}

for dia, ids_must_go in zip(dias_coleta, must_go_por_dia):
    dia_str = str(dia)[:10]
    must_go_dict[dia_str] = ids_must_go

# Transformar o dicionário em DataFrame usando transposição
max_len = max(len(ids) for ids in must_go_dict.values())
must_go_df_transposed = pd.DataFrame({k: v + [None]*(max_len - len(v)) for k, v in must_go_dict.items()})

# Salvar nova versão no Excel
output_path_transposed = "resultados_lookahead_mustgo_transposto.xlsx"
with pd.ExcelWriter(output_path_transposed) as writer:
    must_go_df_transposed.to_excel(writer, sheet_name="must_go_ids_por_dia", index=False)


output_path_completo = "matriz_enchimento_completa.xlsx"
df_niveis.to_excel(output_path_completo, index=False)

output_path_transposed, output_path_completo


('resultados_lookahead_mustgo_transposto.xlsx',
 'matriz_enchimento_completa.xlsx')